In [35]:
#%pip install "routellm[serve,eval]"


**Script de Introdução ao Roteamento de LLMs com o RouteLLM**

O avanço dos Modelos de Linguagem de Grande Porte (LLMs) tem revolucionado a forma como interagimos com sistemas de inteligência artificial. No entanto, modelos mais poderosos, como o GPT-4, são computacionalmente intensivos e caros, enquanto modelos menores podem não oferecer o mesmo nível de desempenho. Para equilibrar custo e qualidade, surge o conceito de **roteamento de LLMs**, onde diferentes modelos são utilizados com base na complexidade da consulta.

O **RouteLLM** é um framework que implementa esse conceito, permitindo que as aplicações decidam automaticamente qual modelo utilizar para cada consulta, otimizando tanto a eficiência quanto a qualidade das respostas.

---

**Análise Conceitual do Script**

Vamos analisar o script fornecido para entender como o roteamento de LLMs é implementado com o RouteLLM.

```python
import os
import yaml
from routellm.controller import Controller

# Define a chave da API da OpenAI
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# Define os modelos forte e fraco
strong_model = "gpt-4o"      # Modelo forte (mais capaz, custo mais alto)
weak_model = "gpt-4o-mini"   # Modelo fraco (menos capaz, custo mais baixo)
```

1. **Configuração Inicial**

   - **Importações**: Carregamos as bibliotecas necessárias, incluindo `os` para manipular variáveis de ambiente, `yaml` para lidar com arquivos de configuração e `Controller` do RouteLLM para gerenciar o roteamento.
   - **Chave de API**: Definimos a chave da API da OpenAI para autenticação nas chamadas aos modelos.

2. **Definição dos Modelos**

   - **Modelos**: Especificamos o modelo forte (`gpt-4o`) e o modelo fraco (`gpt-4o-mini`).
     - **Modelo Forte**: Mais poderoso, capaz de lidar com consultas complexas, mas com custo operacional maior.
     - **Modelo Fraco**: Menos potente, adequado para consultas simples, com menor custo.

```python
# Carrega a configuração do arquivo YAML
try:
    with open('config.example.yaml', 'r') as f:
        config = yaml.safe_load(f)
except FileNotFoundError:
    print("Arquivo de configuração 'config.example.yaml' não encontrado. Usando a configuração padrão.")
    config = None
```

3. **Carregamento da Configuração**

   - **Configuração**: Tentamos carregar parâmetros adicionais de configuração a partir de um arquivo YAML.
   - **Fallback**: Se o arquivo não for encontrado, usamos as configurações padrão embutidas no RouteLLM.

```python
# Inicializa o Controller com o roteador MF
client = Controller(
    routers=["mf"],          # Usa o roteador de Fatoração de Matrizes
    strong_model=strong_model,
    weak_model=weak_model,
    config=config
)
```

4. **Inicialização do Controller**

   - **Controller**: Criamos uma instância do `Controller`, especificando o uso do roteador **mf**.
   - **Roteador MF**: Utiliza um modelo baseado em Fatoração de Matrizes para decidir qual modelo (forte ou fraco) deve responder a cada consulta, com base em dados de preferência e características dos prompts.

```python
# Define o threshold para decisões de roteamento
threshold = 0.5  # Quanto menor o threshold, mais o modelo forte é utilizado
```

5. **Definição do Threshold**

   - **Threshold**: Valor que determina o ponto de corte para usar o modelo forte.
     - **Threshold Baixo**: O modelo forte é usado com mais frequência.
     - **Threshold Alto**: O modelo fraco é preferido, economizando recursos.
   - **Valor Atual**: Com `threshold = 0.5`, o modelo forte é usado quando há pelo menos 50% de chance de oferecer uma resposta significativamente melhor.

```python
def get_response(prompt):
    # Gera a resposta usando o RouteLLM
    response = client.chat.completions.create(
        model=f"router-mf-{threshold}",  # Especifica o roteador e o threshold
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    model_used = response.model  # Obtém o modelo que foi utilizado
    content = response.choices[0].message.content
    return model_used, content
```

6. **Função para Obter Respostas**

   - **get_response**: Função que envia o prompt ao RouteLLM e retorna a resposta junto com o modelo utilizado.
   - **Chamadas ao Modelo**: Usamos o método `create` para gerar uma resposta, especificando o modelo no formato `router-mf-{threshold}`.

```python
prompts = [
    "Descreva a equação de Navier-Stokes",
    "Qual o maior país do mundo em extensão territorial?",
    "Calcule o vórtice de Burgers–Rott",
    "Qual é a capital do Brasil?"
]
```

7. **Lista de Prompts**

   - **Diversidade de Consultas**: Incluímos prompts de diferentes níveis de complexidade para testar o roteamento.
     - **Consultas Complexas**: Podem exigir o modelo forte.
     - **Consultas Simples**: Podem ser adequadamente respondidas pelo modelo fraco.

```python
for prompt in prompts:
    try:
        model_used, response = get_response(prompt)
        print(f"Prompt: {prompt}")
        print(f"Modelo usado: {model_used}")
        print(f"Resposta: {response}\n")
    except Exception as e:
        print(f"Ocorreu um erro ao processar o prompt: {prompt}")
        print(f"Erro: {e}\n")
```

8. **Processamento dos Prompts**

   - **Iteração**: Para cada prompt, obtemos a resposta e identificamos qual modelo foi usado.
   - **Exibição**: Imprimimos o prompt, o modelo utilizado e a resposta correspondente.
   - **Tratamento de Erros**: Capturamos exceções para garantir que o script continue executando mesmo se ocorrer algum erro.

---

**Conceitos Fundamentais de LLM Routing com RouteLLM**

1. **Roteamento Baseado em Preferência**

   - **Objetivo**: Alocar recursos computacionais de forma eficiente, usando modelos mais caros apenas quando necessário.
   - **Decisão Automatizada**: O roteador determina qual modelo usar com base na previsão de qual fornecerá a melhor resposta para o prompt dado.

2. **Funcionamento do Roteador MF**

   - **Fatoração de Matrizes**: Técnica de aprendizado de máquina que decompõe interações complexas em fatores latentes.
   - **Dados de Treinamento**: O modelo é treinado em dados onde humanos avaliaram qual modelo oferece melhores respostas para diferentes prompts.
   - **Predição da Taxa de Vitória**: Calcula a probabilidade de o modelo forte superar o modelo fraco para o prompt atual.

3. **Threshold e Tomada de Decisão**

   - **Threshold (Limite)**: Valor que serve como referência para decidir qual modelo usar.
   - **Interpretação**:
     - **Taxa de Vitória ≥ Threshold**: Usa o modelo forte.
     - **Taxa de Vitória < Threshold**: Usa o modelo fraco.
   - **Ajuste Estratégico**: O threshold pode ser ajustado para equilibrar custo e desempenho conforme as necessidades do sistema.

4. **Benefícios do Roteamento**

   - **Eficiência de Custos**: Reduz o uso desnecessário de modelos caros para consultas simples.
   - **Manutenção da Qualidade**: Garante que consultas complexas recebam respostas de alta qualidade.
   - **Escalabilidade**: Permite lidar com um grande volume de consultas de maneira otimizada.

---

**Exemplos Práticos do Script**

- **Prompt 1**: *"Descreva a equação de Navier-Stokes"*
  - **Análise**: Tópico avançado em mecânica dos fluidos.
  - **Expectativa**: O roteador provavelmente usará o modelo forte (`gpt-4o`) para fornecer uma explicação detalhada.

- **Prompt 2**: *"Qual o maior país do mundo em extensão territorial?"*
  - **Análise**: Pergunta factual simples.
  - **Expectativa**: O modelo fraco (`gpt-4o-mini`) deve ser suficiente para responder corretamente.

- **Prompt 3**: *"Calcule o vórtice de Burgers–Rott"*
  - **Análise**: Requer conhecimento especializado e cálculos complexos.
  - **Expectativa**: O modelo forte será selecionado para lidar com a complexidade.

- **Prompt 4**: *"Qual é a capital do Brasil?"*
  - **Análise**: Informação geral e direta.
  - **Expectativa**: O modelo fraco pode responder adequadamente.



In [36]:
import os
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

# Configuração da chave de API da OpenAI
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["GROQ_API_KEY"] = os.getenv('GROQ_API_KEY')

In [37]:
import os
import yaml
from routellm.controller import Controller

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# Define the strong and weak models
strong_model = "gpt-4o"      # Strong model (more capable, higher cost)
weak_model = "gpt-4o-mini"   # Weak model (less capable, lower cost)

# Load the config from the YAML file
try:
    with open('config.example.yaml', 'r') as f:
        config = yaml.safe_load(f)
except FileNotFoundError:
    print("Configuration file 'config.example.yaml' not found. Using default configuration.")
    config = None

# Initialize the Controller with the MF router
client = Controller(
    routers=["mf"],          # Use the Matrix Factorization router
    strong_model=strong_model,
    weak_model=weak_model,
    config=config
)

# Set the threshold for routing decisions
threshold = 0.3  # The lower the threshold, the more the strong model is used

def get_response(prompt):
    # Generate response using RouteLLM
    response = client.chat.completions.create(
        model=f"router-mf-{threshold}",  # Specify router and threshold
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    model_used = response.model  # Get the model that was used
    content = response.choices[0].message.content
    return model_used, content

prompts = [
    "Descreva a equação de Navier-Stokes",
    "Qual o maior país do mundo em extensão territorial?",
    "Calcule o vórtice de Burgers–Rott",
    "Qual é a capital do Brasil?"
]

for prompt in prompts:
    try:
        model_used, response = get_response(prompt)
        print(f"Prompt: {prompt}")
        print(f"Model used: {model_used}")
        print(f"Response: {response}\n")
    except Exception as e:
        print(f"An error occurred while processing the prompt: {prompt}")
        print(f"Error: {e}\n")


Configuration file 'config.example.yaml' not found. Using default configuration.
Prompt: Descreva a equação de Navier-Stokes
Model used: gpt-4o-2024-08-06
Response: A equação de Navier-Stokes descreve o movimento de fluidos. É uma equação fundamental em mecânica dos fluidos e tem ampla aplicação em engenharia, meteorologia, oceanografia e muitos outros campos. A equação é baseada na aplicação das leis de conservação de massa, momento e energia a um volume de controle no fluido. A forma mais geral da equação de Navier-Stokes para um fluido incompressível e newtoniano é a seguinte:

\[
\rho \left( \frac{\partial \mathbf{u}}{\partial t} + (\mathbf{u} \cdot \nabla) \mathbf{u} \right) = -\nabla p + \mu \nabla^2 \mathbf{u} + \mathbf{f}
\]

Onde:
- \(\mathbf{u}\) é o vetor velocidade do fluido.
- \(t\) é o tempo.
- \(\rho\) é a densidade do fluido.
- \(p\) é a pressão do fluido.
- \(\mu\) é a viscosidade dinâmica do fluido.
- \(\nabla\) é o operador nabla (que representa gradiente).
- \(\nabl

**Conclusão**

O script exemplifica como implementar o roteamento de LLMs usando o RouteLLM para otimizar a utilização de modelos de linguagem. Ao empregar o roteador de Fatoração de Matrizes, o sistema pode tomar decisões inteligentes sobre qual modelo utilizar para cada consulta, baseando-se em previsões estatísticas e dados de preferência.

**Vantagens Chave:**

- **Custo-Efetividade**: Uso eficiente de recursos computacionais, reduzindo despesas operacionais.
- **Qualidade das Respostas**: Mantém alto padrão de qualidade onde é mais necessário.
- **Flexibilidade**: Possibilidade de ajustar o threshold para atender a diferentes prioridades (custo vs. desempenho).

Este conceito é especialmente relevante em aplicações em que o volume de consultas é grande e variado, permitindo escalabilidade sem comprometer a experiência do usuário.


**Considerações Finais**

O roteamento de LLMs com o RouteLLM representa uma abordagem estratégica para maximizar o valor obtido de modelos de linguagem, alinhando capacidades técnicas com objetivos de negócio. Compreender e aplicar esses conceitos pode levar ao desenvolvimento de sistemas mais inteligentes, eficientes e sustentáveis.

Se houver interesse em aprofundar em aspectos específicos ou em como adaptar esse framework a outras necessidades, estou à disposição para auxiliar!